In [9]:
import wrds
db = wrds.Connection()

Loading library list...
Done


In [10]:
sql_query = """
    SELECT date, permno, ret, prc, vol
    FROM crsp.dsf
    WHERE permno = 14593
    AND date >= '2024-01-01'
    AND date <= '2024-12-31'
"""

aapl_data = db.raw_sql(sql_query)

In [11]:
aapl_data.head()
# print(aapl_data.info())


,date,permno,ret,prc,vol
0,2024-01-02,14593,-0.035787,185.64,81752737.0
1,2024-01-03,14593,-0.007488,184.25,58136569.0
2,2024-01-04,14593,-0.0127,181.91,71280275.0
3,2024-01-05,14593,-0.004013,181.17999,62064040.0
4,2024-01-08,14593,0.024175,185.56,58748031.0


In [12]:
query_crsp = """
    SELECT date, permno, 
           abs(prc) * shrout AS mktcap, 
           ret
    FROM crsp.dsf
    WHERE permno = 14593 AND date >= '2023-01-01'
"""
crsp_data = db.raw_sql(query_crsp)

In [13]:
crsp_data

,date,permno,mktcap,ret
0,2023-01-03,14593,1981409843.49,-0.037405
1,2023-01-04,14593,2001846548.52,0.010314
2,2023-01-05,14593,1980617723.14,-0.010605
3,2023-01-06,14593,2053492795.34,0.036794
4,2023-01-09,14593,2061082963.58787,0.004089
...,...,...,...,...
497,2024-12-24,14593,3902905649.75823,0.011478
498,2024-12-26,14593,3915300322.30177,0.003176
499,2024-12-27,14593,3863453200.57,-0.013242
500,2024-12-30,14593,3812210560.6,-0.013263


In [17]:
# Book Value and Profitability
query_comp = """
    SELECT datadate, gvkey, tic,
           ceq AS book_value, 
           ni / NULLIF(at, 0) AS roa, 
           ni / NULLIF(ceq, 0) AS roe
    FROM comp.funda
    WHERE tic = 'AAPL' 
      AND indfmt = 'INDL' 
      AND datafmt = 'STD' 
      AND popsrc = 'D' 
      AND consol = 'C'
      AND datadate >= '2023-01-01'
"""
comp_data = db.raw_sql(query_comp)


In [18]:
comp_data

,datadate,gvkey,tic,book_value,roa,roe
0,2023-09-30,001690,AAPL,62146.0,0.275098,1.56076
1,2024-09-30,001690,AAPL,56950.0,0.256825,1.645935
2,2025-09-30,001690,AAPL,73733.0,0.311796,1.51913


In [19]:
# Beta and Volatility (Market Model)
query_beta = """
    SELECT date, permno, b_mkt, res_var
    FROM wrdsapps.beta_daily
    WHERE permno = 14593 AND date >= '2023-01-01'
"""
beta_data = db.raw_sql(query_beta)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "wrdsapps.beta_daily" does not exist
LINE 3:     FROM wrdsapps.beta_daily
                 ^

[SQL: 
    SELECT date, permno, b_mkt, res_var
    FROM wrdsapps.beta_daily
    WHERE permno = 14593 AND date >= '2023-01-01'
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
beta_data

In [16]:
# Pulling pre-calculated ratios directly
query_ratios = """
    SELECT public_date, bm, roe, mom12m, capital_ratio
    FROM wrdsapps.firm_ratio
    WHERE permno = 14593 AND public_date >= '2023-01-01'
"""
ratios_df = db.raw_sql(query_ratios)
ratios_df

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "mom12m" does not exist
LINE 2:     SELECT public_date, bm, roe, mom12m, capital_ratio
                                         ^

[SQL: 
    SELECT public_date, bm, roe, mom12m, capital_ratio
    FROM wrdsapps.firm_ratio
    WHERE permno = 14593 AND public_date >= '2023-01-01'
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [24]:
import wrds
import pandas as pd
import numpy as np

# Connect to WRDS
db = wrds.Connection()

# 1. Pull Monthly Returns, Market Cap, and Market Benchmark (CRSP)
# 'vwretd' is the value-weighted market return for beta calculation
crsp_query = """
    SELECT a.date, a.permno, a.ret, 
           abs(a.prc) * a.shrout AS mktcap, 
           b.vwretd AS mkt_ret
    FROM crsp.msf AS a
    LEFT JOIN crsp.msi AS b ON a.date = b.date
    WHERE a.date >= '2000-01-01' AND a.date <= '2025-12-31'
"""
df = db.raw_sql(crsp_query, date_cols=['date'])

# 2. Calculate Momentum (Past 12-month return, skipping month t)
# We use log returns for easier aggregation across time
df = df.sort_values(['permno', 'date'])
df['log_ret'] = np.log(1 + df['ret'].fillna(0))
df['momentum'] = df.groupby('permno')['log_ret'].transform(lambda x: x.rolling(11).sum().shift(2))
df['momentum'] = np.exp(df['momentum']) - 1

# 3. Calculate Rolling Volatility and Beta (Manual Calculation)
# 12-month rolling standard deviation
df['volatility'] = df.groupby('permno')['ret'].transform(lambda x: x.rolling(12).std())

# Rolling Beta (Covariance of Stock & Market / Variance of Market)
def calc_rolling_beta(group):
    cov = group['ret'].rolling(12).cov(group['mkt_ret'])
    mkt_var = group['mkt_ret'].rolling(12).var()
    return cov / mkt_var

df['beta'] = df.groupby('permno', group_keys=False).apply(calc_rolling_beta)

# 4. Pull Book Value & Profitability (Compustat via CCM Link Table)
# 'lpermno' is the standard linked identifier in crsp.ccmxpf_linktable
comp_query = """
    SELECT a.lpermno AS permno, b.datadate, b.ceq, b.ni, b.at, b.tic
    FROM crsp.ccmxpf_linktable AS a
    INNER JOIN comp.funda AS b ON a.gvkey = b.gvkey
    WHERE a.linktype IN ('LU', 'LC') AND a.linkprim IN ('P', 'C')
      AND b.indfmt='INDL' AND b.datafmt='STD' AND b.popsrc='D' AND b.consol='C'
"""
comp = db.raw_sql(comp_query, date_cols=['datadate'])

# 5. Align and Merge
# Match annual accounting data to the following year's monthly returns
df['year'] = df['date'].dt.year
comp['year'] = comp['datadate'].dt.year + 1 

final_df = pd.merge(df, comp, on=['permno', 'year'], how='left')

# Final Calculations
final_df['book_to_market'] = final_df['ceq'] / (final_df['mktcap'] / 1000) # CEQ is in millions
final_df['roa'] = final_df['ni'] / final_df['at']

# Standardize variables cross-sectionally (Z-score by date)
for col in ['mktcap', 'book_to_market', 'momentum', 'volatility', 'beta', 'roa']:
    final_df[col] = final_df.groupby('date')[col].transform(lambda x: (x - x.mean()) / x.std())

# Clean and filter
final_df = final_df.dropna(subset=['ret', 'momentum', 'beta'])
print(f"Successfully pulled {len(final_df)} stock-month observations.")


Loading library list...
Done


/var/folders/qm/q59chcx17clc7f88_brlpd8h0000gn/T/ipykernel_26443/3303150019.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['beta'] = df.groupby('permno', group_keys=False).apply(calc_rolling_beta)
/Users/harrisonzhang/anaconda3/envs/env3119/lib/python3.11/site-packages/numpy/_core/_methods.py:132: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/harrisonzhang/anaconda3/envs/env3119/lib/python3.11/site-packages/numpy/_core/_methods.py:132: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/harrisonzhang/anaconda3/envs/env3119/lib/python3.11/site-pac

Successfully pulled 2106305 stock-month observations.


In [26]:
final_df

,date,permno,ret,mktcap,mkt_ret,log_ret,momentum,volatility,beta,year,datadate,ceq,ni,at,tic,book_to_market,roa
12,2001-01-31,10001,0.012821,-0.159462,0.039573,0.01274,0.457600,-1.022008,-0.785263,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.038421,<NA>
13,2001-02-28,10001,-0.012658,-0.156688,-0.099084,-0.012739,0.570706,-1.064971,-0.735675,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.038591,<NA>
14,2001-03-30,10001,0.038462,-0.15767,-0.070408,0.037741,0.538049,-1.069359,-0.715246,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.039324,<NA>
15,2001-04-30,10001,-0.025,-0.155612,0.083834,-0.025318,0.658569,-1.070044,-0.801179,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.044107,<NA>
16,2001-05-31,10001,0.097436,-0.157219,0.010442,0.092977,0.683808,-1.055869,-0.816090,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.042225,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480281,2024-08-30,93436,-0.077391,9.755968,0.021572,-0.08055,-0.531164,0.078537,0.216912,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.128659,<NA>
2480282,2024-09-30,93436,0.221942,11.772103,0.020969,0.200441,-0.391592,0.171307,0.323071,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.131128,<NA>
2480283,2024-10-31,93436,-0.045025,11.321767,-0.008298,-0.04607,-0.547516,0.077844,0.082367,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.125883,<NA>
2480284,2024-11-29,93436,0.381469,14.88241,0.064855,0.323147,0.066975,0.200661,0.288641,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.128468,<NA>


In [28]:
# 1. Sort to ensure we can calculate 'Asset Growth' for Investment
final_df = final_df.sort_values(['permno', 'date'])

# 2. Calculate Profitability (ROA)
# Ensure we use NULLIF or check for zeros in 'at'
final_df['roa'] = final_df['ni'] / final_df['at']

# 3. Calculate Investment (Asset Growth)
# We shift by 12 months to get the previous year's assets for that specific stock
final_df['at_lag'] = final_df.groupby('permno')['at'].shift(12)
final_df['investment'] = (final_df['at'] - final_df['at_lag']) / final_df['at_lag']

# 4. Forward-fill accounting data
# Accounting data is annual, so we carry the values forward for all 12 months of the year
final_df[['roa', 'investment']] = final_df.groupby('permno')[['roa', 'investment']].ffill()

final_df

,date,permno,ret,mktcap,mkt_ret,log_ret,momentum,volatility,beta,year,datadate,ceq,ni,at,tic,book_to_market,roa,at_lag,investment
12,2001-01-31,10001,0.012821,-0.159462,0.039573,0.01274,0.457600,-1.022008,-0.785263,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.038421,0.025656,<NA>,<NA>
13,2001-02-28,10001,-0.012658,-0.156688,-0.099084,-0.012739,0.570706,-1.064971,-0.735675,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.038591,0.025656,<NA>,<NA>
14,2001-03-30,10001,0.038462,-0.15767,-0.070408,0.037741,0.538049,-1.069359,-0.715246,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.039324,0.025656,<NA>,<NA>
15,2001-04-30,10001,-0.025,-0.155612,0.083834,-0.025318,0.658569,-1.070044,-0.801179,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.044107,0.025656,<NA>,<NA>
16,2001-05-31,10001,0.097436,-0.157219,0.010442,0.092977,0.683808,-1.055869,-0.816090,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.042225,0.025656,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480281,2024-08-30,93436,-0.077391,9.755968,0.021572,-0.08055,-0.531164,0.078537,0.216912,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.128659,0.140661,82338.0,0.294882
2480282,2024-09-30,93436,0.221942,11.772103,0.020969,0.200441,-0.391592,0.171307,0.323071,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.131128,0.140661,82338.0,0.294882
2480283,2024-10-31,93436,-0.045025,11.321767,-0.008298,-0.04607,-0.547516,0.077844,0.082367,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.125883,0.140661,82338.0,0.294882
2480284,2024-11-29,93436,0.381469,14.88241,0.064855,0.323147,0.066975,0.200661,0.288641,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.128468,0.140661,82338.0,0.294882


In [31]:
import pandas as pd
import numpy as np

# 1. Calculate Profitability (ROA)
# Using ni (Net Income) and at (Total Assets) from your merged Compustat data
final_df['roa'] = final_df['ni'] / final_df['at']

# 2. Calculate Investment (Asset Growth)
# Requires sorting by stock and date to find the previous year's assets
final_df = final_df.sort_values(['permno', 'date'])
final_df['at_lag'] = final_df.groupby('permno')['at'].shift(12)
final_df['investment'] = (final_df['at'] - final_df['at_lag']) / final_df['at_lag']

# 3. Clean up the dataframe
# Removing intermediate columns if desired
final_df = final_df.drop(columns=['at_lag'])
final_df

,date,permno,ret,mktcap,mkt_ret,log_ret,momentum,volatility,beta,year,datadate,ceq,ni,at,tic,book_to_market,roa,investment
12,2001-01-31,10001,0.012821,-0.159462,0.039573,0.01274,0.457600,-1.022008,-0.785263,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.038421,0.025656,<NA>
13,2001-02-28,10001,-0.012658,-0.156688,-0.099084,-0.012739,0.570706,-1.064971,-0.735675,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.038591,0.025656,<NA>
14,2001-03-30,10001,0.038462,-0.15767,-0.070408,0.037741,0.538049,-1.069359,-0.715246,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.039324,0.025656,<NA>
15,2001-04-30,10001,-0.025,-0.155612,0.083834,-0.025318,0.658569,-1.070044,-0.801179,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.044107,0.025656,<NA>
16,2001-05-31,10001,0.097436,-0.157219,0.010442,0.092977,0.683808,-1.055869,-0.816090,2001,2000-06-30,13.961,1.297,50.553,EGAS,-0.042225,0.025656,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480281,2024-08-30,93436,-0.077391,9.755968,0.021572,-0.08055,-0.531164,0.078537,0.216912,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.128659,0.140661,0.294882
2480282,2024-09-30,93436,0.221942,11.772103,0.020969,0.200441,-0.391592,0.171307,0.323071,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.131128,0.140661,0.294882
2480283,2024-10-31,93436,-0.045025,11.321767,-0.008298,-0.04607,-0.547516,0.077844,0.082367,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.125883,0.140661,0.294882
2480284,2024-11-29,93436,0.381469,14.88241,0.064855,0.323147,0.066975,0.200661,0.288641,2024,2023-12-31,62634.0,14997.0,106618.0,TSLA,-0.128468,0.140661,0.294882


In [32]:
# Install pyarrow if you haven't already: !pip install pyarrow
# Save the completed dataframe to a parquet file
final_df.to_parquet('wrds_stock_characteristics.parquet', engine='pyarrow', index=False)

print("Data successfully saved to wrds_stock_characteristics.parquet")

Data successfully saved to wrds_stock_characteristics.parquet


In [30]:
print(len(final_df["permno"].unique().tolist()))

20975


In [29]:
tmp = db.raw_sql("""
    select gvkey, datadate, at, ceq, ni, sale, seq, txdb
    from comp.funda
    where datadate between '2009-01-01' and '2025-12-31'
""")

tmp


,gvkey,datadate,at,ceq,ni,sale,seq,txdb
0,001004,2009-05-31,1377.511,656.895,78.651,1423.976,656.895,40.263
1,001004,2010-05-31,1501.042,746.906,44.628,1352.151,746.906,57.335
2,001004,2011-05-31,1703.727,835.845,69.826,1775.782,835.845,98.322
3,001004,2011-05-31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,001004,2012-05-31,2195.653,864.649,67.723,2074.498,864.649,115.908
...,...,...,...,...,...,...,...,...
328551,369350,2025-12-31,8791.851,733.828,344.019,9287.332,733.828,241.87
328552,370994,2023-12-31,3989.901,269.582,<NA>,<NA>,269.582,40.041
328553,370994,2023-12-31,3989.901,269.582,86.971,1034.425,269.582,40.041
328554,370994,2024-12-31,4772.024,381.906,<NA>,<NA>,381.906,33.445
